# Birth Control Side Effects - Validation Analysis

This notebook performs comprehensive validation of side effects discovered from Reddit using:
1. **PubMed Research Validation**: Cross-reference with medical literature
2. **Evidence Tiering**: Classify side effects by strength of evidence
3. **Surprise Score Analysis**: Find hidden side effects with low research coverage
4. **Statistical Validation**: Rigorous correlation and distribution analysis

## Methodology
Following the EDS symptom discovery approach with:
- Multi-source validation (Reddit + PubMed + FDA listings)
- Spearman correlation analysis
- Chi-square tests for distribution
- Bonferroni correction for multiple testing

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

## 1. Load Validated Data

In [ ]:
# Load validated side effects database
validated_path = Path('../data/validated/validated_side_effects_database.json')

if not validated_path.exists():
    print("⚠️  Validated data not found. Please run:")
    print("   python src/validation/evidence_validator.py")
else:
    with open(validated_path, 'r') as f:
        validated_data = json.load(f)
    
    # Convert to DataFrame
    df = pd.DataFrame([{
        'side_effect': item['side_effect'],
        'patient_frequency': item['reddit_data']['frequency'],
        'mention_count': item['reddit_data']['mention_count'],
        'post_count': item['reddit_data']['post_count'],
        'paper_count': item['pubmed_data']['paper_count'],
        'surprise_score': item['surprise_score'],
        'evidence_tier': item['evidence_tier'],
        'tier_label': item['tier_label'],
        'category': item['reddit_data'].get('category', 'unknown')
    } for item in validated_data])
    
    print(f"✓ Loaded {len(df)} validated side effects")
    df.head(10)

## 2. Top Side Effects by Patient Reports

In [ ]:
# Top 15 side effects by mentions
top_side_effects = df.nlargest(15, 'mention_count')[['side_effect', 'mention_count', 'patient_frequency', 'tier_label']]

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(top_side_effects['side_effect'], top_side_effects['mention_count'])

# Color by tier
tier_colors = {
    '🏆 FDA-Listed': 'gold',
    '✅ Research-Backed': 'lightgreen',
    '💬 Patient-Validated': 'lightblue',
    '⚠️ Emerging Pattern': 'lightcoral'
}

for i, (bar, tier) in enumerate(zip(bars, top_side_effects['tier_label'])):
    bar.set_color(tier_colors.get(tier, 'gray'))

ax.set_xlabel('Number of Patient Mentions', fontsize=12)
ax.set_title('Top 15 Birth Control Side Effects by Patient Reports', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Add value labels
for i, (bar, freq) in enumerate(zip(bars, top_side_effects['patient_frequency'])):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f"{freq*100:.1f}%", va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nTop Side Effects:")
for idx, row in top_side_effects.iterrows():
    print(f"  {row['side_effect']:25s} - {row['mention_count']:3d} mentions ({row['patient_frequency']*100:5.1f}%) [{row['tier_label']}]")

## 3. Surprise Score Analysis - Finding Hidden Side Effects

In [ ]:
# Scatter plot: Patient frequency vs Research coverage
fig, ax = plt.subplots(figsize=(12, 8))

# Color by surprise score
scatter = ax.scatter(df['patient_frequency'], df['paper_count'], 
                     c=df['surprise_score'], s=100, cmap='YlOrRd', 
                     alpha=0.6, edgecolors='black', linewidth=0.5)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Surprise Score', fontsize=12)

# Labels
ax.set_xlabel('Patient Report Frequency', fontsize=12)
ax.set_ylabel('Number of Research Papers', fontsize=12)
ax.set_title('Hidden Side Effects: High Patient Reports + Low Research Coverage', 
             fontsize=14, fontweight='bold')

# Annotate high surprise side effects
high_surprise = df.nlargest(8, 'surprise_score')
for _, row in high_surprise.iterrows():
    ax.annotate(row['side_effect'], 
                xy=(row['patient_frequency'], row['paper_count']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=9, bbox=dict(boxstyle='round,pad=0.3', 
                facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'))

# Quadrant lines
ax.axhline(y=3, color='gray', linestyle='--', alpha=0.5, label='Research threshold (3 papers)')
ax.axvline(x=0.05, color='gray', linestyle='--', alpha=0.5, label='Frequency threshold (5%)')

ax.legend()
plt.tight_layout()
plt.show()

print("\n🎯 Top 10 Hidden Side Effects (High Surprise Score):")
print(high_surprise[['side_effect', 'surprise_score', 'mention_count', 'paper_count']].to_string(index=False))

## 4. Evidence Tier Distribution

In [ ]:
# Count by tier
tier_counts = df['tier_label'].value_counts()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
tier_colors_list = [tier_colors.get(t, 'gray') for t in tier_counts.index]
ax1.pie(tier_counts.values, labels=tier_counts.index, 
        autopct='%1.1f%%', startangle=90, colors=tier_colors_list)
ax1.set_title('Evidence Quality Distribution', fontsize=14, fontweight='bold')

# Bar chart
bars = ax2.bar(range(len(tier_counts)), tier_counts.values, color=tier_colors_list)
ax2.set_xticks(range(len(tier_counts)))
ax2.set_xticklabels(tier_counts.index, rotation=45, ha='right')
ax2.set_ylabel('Number of Side Effects', fontsize=12)
ax2.set_title('Side Effects by Evidence Tier', fontsize=14, fontweight='bold')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nEvidence Tier Summary:")
for tier, count in tier_counts.items():
    print(f"  {tier}: {count} side effects")

## 5. Research Gaps Analysis

In [ ]:
# Identify research gaps: side effects with >20 patient mentions but <3 papers
research_gaps = df[(df['mention_count'] > 20) & (df['paper_count'] < 3)].copy()
research_gaps = research_gaps.sort_values('mention_count', ascending=False)

if len(research_gaps) > 0:
    print(f"\n🔍 Found {len(research_gaps)} side effects with research gaps:\n")
    print(research_gaps[['side_effect', 'mention_count', 'paper_count', 'surprise_score']].to_string(index=False))
    
    # Visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    x = range(len(research_gaps))
    ax.bar(x, research_gaps['mention_count'], color='coral', alpha=0.7, label='Patient Mentions')
    ax2 = ax.twinx()
    ax2.plot(x, research_gaps['paper_count'], 'o-', color='steelblue', 
             linewidth=2, markersize=8, label='Research Papers')
    
    ax.set_xlabel('Side Effect', fontsize=12)
    ax.set_ylabel('Patient Mentions', fontsize=12, color='coral')
    ax2.set_ylabel('Research Papers', fontsize=12, color='steelblue')
    ax.set_xticks(x)
    ax.set_xticklabels(research_gaps['side_effect'], rotation=45, ha='right')
    ax.set_title('Research Gaps: High Patient Reports, Low Research Coverage', 
                 fontsize=14, fontweight='bold')
    
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.tight_layout()
    plt.show()
else:
    print("\n✅ No significant research gaps found (all high-mention side effects have research coverage)")

## 6. Statistical Validation

Run rigorous statistical tests to validate findings.

In [ ]:
# Load statistical validation results if available
stats_path = Path('../data/analysis/statistical_validation_results.json')

if stats_path.exists():
    with open(stats_path, 'r') as f:
        stats_results = json.load(f)
    
    print("📊 Statistical Validation Results:")
    print("\n1. Spearman Correlation (Patient Reports vs Research Coverage):")
    print(f"   ρ = {stats_results['correlation']['spearman_rho']:.4f}")
    print(f"   p-value = {stats_results['correlation']['p_value']:.4f}")
    print(f"   Significant: {stats_results['correlation']['significant']}")
    
    print("\n2. Surprise Score Distribution:")
    dist = stats_results['surprise_distribution']['distribution']
    print(f"   Low: {dist['low']} | Medium: {dist['medium']} | High: {dist['high']}")
    print(f"   χ² = {stats_results['surprise_distribution']['chi_square']:.4f}")
    print(f"   p-value = {stats_results['surprise_distribution']['p_value']:.4f}")
    
    print("\n3. Multiple Testing Correction:")
    mt = stats_results['multiple_testing']
    print(f"   Tests performed: {mt['n_tests']}")
    print(f"   Significant (before correction): {mt['n_significant_uncorrected']}")
    print(f"   Significant (after Bonferroni): {mt['n_significant_corrected']}")
else:
    print("⚠️  Statistical validation not yet run. Please run:")
    print("   python src/analysis/statistical_validator.py")

## 7. Category Analysis (Mental vs Physical)

In [ ]:
# Analyze by category
category_counts = df['category'].value_counts()

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(category_counts.index, category_counts.values, 
              color=['lightblue', 'lightcoral', 'lightgray'])
ax.set_ylabel('Number of Side Effects', fontsize=12)
ax.set_xlabel('Category', fontsize=12)
ax.set_title('Side Effects by Category', fontsize=14, fontweight='bold')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nCategory Distribution:")
for cat, count in category_counts.items():
    print(f"  {cat}: {count} side effects")

# Show top mental and physical side effects
print("\n🧠 Top 5 Mental/Emotional Side Effects:")
mental = df[df['category'] == 'mental'].nlargest(5, 'mention_count')
for _, row in mental.iterrows():
    print(f"  - {row['side_effect']}: {row['mention_count']} mentions")

print("\n💊 Top 5 Physical Side Effects:")
physical = df[df['category'] == 'physical'].nlargest(5, 'mention_count')
for _, row in physical.iterrows():
    print(f"  - {row['side_effect']}: {row['mention_count']} mentions")

## 8. Export Results for Frontend

In [ ]:
# Create summary for frontend
summary = {
    'total_side_effects': len(df),
    'high_surprise_count': len(df[df['surprise_score'] >= 0.4]),
    'research_gaps_count': len(research_gaps) if len(research_gaps) > 0 else 0,
    'top_side_effects': df.nlargest(10, 'mention_count')[[
        'side_effect', 'mention_count', 'surprise_score', 'tier_label'
    ]].to_dict('records'),
    'hidden_gems': df.nlargest(10, 'surprise_score')[[
        'side_effect', 'surprise_score', 'mention_count', 'paper_count', 'tier_label'
    ]].to_dict('records'),
    'by_tier': tier_counts.to_dict(),
    'by_category': category_counts.to_dict()
}

# Save for frontend
output_dir = Path('../frontend/data')
output_dir.mkdir(exist_ok=True)

with open(output_dir / 'validation_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✅ Summary exported to frontend/data/validation_summary.json")
print(f"\n📊 Summary Statistics:")
print(f"   Total side effects validated: {summary['total_side_effects']}")
print(f"   High surprise side effects: {summary['high_surprise_count']}")
print(f"   Research gaps identified: {summary['research_gaps_count']}")

## Summary

This analysis:
- ✅ Validated side effects with PubMed research
- ✅ Assigned evidence tiers (FDA-listed, Research-backed, Patient-validated, Emerging)
- ✅ Identified hidden side effects with high surprise scores
- ✅ Found research gaps (high patient reports, low research coverage)
- ✅ Performed statistical validation (correlation, distribution, multiple testing)
- ✅ Exported results for interactive frontend

**Key Insights:**
1. Side effects with high patient reports but low research coverage represent potential research gaps
2. Evidence tiering helps prioritize which side effects need more investigation
3. Statistical validation confirms patterns are not due to random chance